In [2]:
pip install openai sentence-transformers numpy requests


   ---------------------------------------- 0.0/275.9 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/275.9 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/275.9 kB ? eta -:--:--
   ------------- ------------------------- 92.2/275.9 kB 880.9 kB/s eta 0:00:01
   ------------- ------------------------- 92.2/275.9 kB 880.9 kB/s eta 0:00:01
   ---------------------- --------------- 163.8/275.9 kB 821.4 kB/s eta 0:00:01
   -------------------------- ----------- 194.6/275.9 kB 845.5 kB/s eta 0:00:01
   -------------------------- ----------- 194.6/275.9 kB 845.5 kB/s eta 0:00:01
   --------------------------------- ---- 245.8/275.9 kB 793.0 kB/s eta 0:00:01
   --------------------------------- ---- 245.8/275.9 kB 793.0 kB/s eta 0:00:01
   --------------------------------- ---- 245.8/275.9 kB 793.0 kB/s eta 0:00:01
   --------------------------------- ---- 245.8/275.9 kB 793.0 kB/s eta 0:00:01
   ------------------------------------ - 266.2/275.9 kB 512.0 k

api_key_r1="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"

## DeepSeek

In [15]:
import gzip
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import numpy as np

# Use API key directly (replace with your actual OpenRouter API key)
api_key = "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"

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

# Load dataset (handles .gz files)
def load_qa_pairs(raw_pool_path):
    qa_pairs = []
    with gzip.open(raw_pool_path, 'rt', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                qa_pairs.append({
                    'question': parts[0],
                    'answers': parts[1:]  # Multiple possible answers
                })
    return qa_pairs

# Load all QA pairs
train_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.train.encoded.gz')
test1_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.test.encoded.gz')
test2_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1000.pool.solr.test.encoded.gz')
all_qa = train_qa + test1_qa + test2_qa

# Encode questions for semantic search
model = SentenceTransformer('all-MiniLM-L6-v2')
questions = [pair['question'] for pair in all_qa]
question_embeddings = model.encode(questions)

def get_insurance_answer(user_query):
    # Find top 3 relevant questions
    query_embedding = model.encode([user_query])
    similarities = np.dot(query_embedding, question_embeddings.T)[0]
    top_indices = np.argsort(similarities)[-3:][::-1]

    # Collect relevant answers
    context_answers = []
    for idx in top_indices:
        context_answers.extend(all_qa[idx]['answers'])

    # Generate response via OpenRouter (R1 Free Model)
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",  # R1 Free model
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert insurance assistant. Provide accurate, concise answers based on the given knowledge.Format the response in Markdown."
                },
                {
                    "role": "user",
                    "content": f"""
                    Question: {user_query}
                    Relevant Answers:
                    {chr(10).join([f"- {ans}" for ans in context_answers])}
                    """
                }
            ],
            temperature=0.3,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test
user_question = "What does comprehensive auto insurance cover?"
answer = get_insurance_answer(user_question)
print(answer)

Comprehensive auto insurance covers non-collision-related damage to your vehicle caused by events outside your control. Key protections include:

### ✅ **Covered Events**  
- **Theft** (vehicle or parts)  
- **Vandalism** (e.g., keying, broken windows)  
- **Natural disasters** (floods, hurricanes, earthquakes)  
- **Weather damage** (hail, falling trees, lightning)  
- **Animal collisions** (e.g., hitting a deer)  
- **Fire/explosions**  
- **Falling objects** (e.g., rocks, debris)  
- **Riots/civil unrest**  

### ❌ **Not Covered**  
- Collisions with other vehicles/objects (covered by **collision insurance**).  
- Medical bills or liability (covered by liability/personal injury coverage).  
- Mechanical failures or wear-and-tear.  
- Personal belongings inside the vehicle (covered by homeowners/renters insurance).  

### 📝 **Key Notes**  
- **Optional coverage** (required if leasing/financing a vehicle).  
- Typically includes a **deductible** (out-of-pocket cost per claim).  
- Doe

In [16]:
from IPython.display import display, Markdown
display(Markdown(answer))


Comprehensive auto insurance covers non-collision-related damage to your vehicle caused by events outside your control. Key protections include:

### ✅ **Covered Events**  
- **Theft** (vehicle or parts)  
- **Vandalism** (e.g., keying, broken windows)  
- **Natural disasters** (floods, hurricanes, earthquakes)  
- **Weather damage** (hail, falling trees, lightning)  
- **Animal collisions** (e.g., hitting a deer)  
- **Fire/explosions**  
- **Falling objects** (e.g., rocks, debris)  
- **Riots/civil unrest**  

### ❌ **Not Covered**  
- Collisions with other vehicles/objects (covered by **collision insurance**).  
- Medical bills or liability (covered by liability/personal injury coverage).  
- Mechanical failures or wear-and-tear.  
- Personal belongings inside the vehicle (covered by homeowners/renters insurance).  

### 📝 **Key Notes**  
- **Optional coverage** (required if leasing/financing a vehicle).  
- Typically includes a **deductible** (out-of-pocket cost per claim).  
- Does not increase liability limits or cover driver injuries.  

Always review your policy for specific inclusions/exclusions.

## Chatbot with loop 

In [20]:
import gzip
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import numpy as np
from IPython.display import display, Markdown


# Use API key directly (replace with your actual OpenRouter API key)
api_key = "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"

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

# Load dataset (handles .gz files)
def load_qa_pairs(raw_pool_path):
    qa_pairs = []
    with gzip.open(raw_pool_path, 'rt', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                qa_pairs.append({
                    'question': parts[0],
                    'answers': parts[1:]  # Multiple possible answers
                })
    return qa_pairs

# Load all QA pairs
train_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.train.encoded.gz')
test1_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1500.pool.solr.test.encoded.gz')
test2_qa = load_qa_pairs(r'D:\NLPInsuranceProject\NLPINSURANCE-FINTECHPROJ\dataset insurance qa\InsuranceQA.question.anslabel.raw.1000.pool.solr.test.encoded.gz')
all_qa = train_qa + test1_qa + test2_qa

# Encode questions for semantic search
model = SentenceTransformer('all-MiniLM-L6-v2')
questions = [pair['question'] for pair in all_qa]
question_embeddings = model.encode(questions)

# Chat history to remember past interactions
chat_history = []

def get_insurance_answer(user_query):
    global chat_history  # Maintain chat memory

    # Encode and find relevant answers
    query_embedding = model.encode([user_query])
    similarities = np.dot(query_embedding, question_embeddings.T)[0]
    top_indices = np.argsort(similarities)[-3:][::-1]
    
    context_answers = []
    for idx in top_indices:
        context_answers.extend(all_qa[idx]['answers'])

    # Add the latest user query to chat history
    chat_history.append({"role": "user", "content": user_query})

    # Generate response via OpenRouter
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {"role": "system", "content": "You are an expert insurance assistant. Provide clear, helpful answers and maintain context."}
            ] + chat_history + [
                {"role": "user", "content": f"""
                Question: {user_query}
                Relevant Answers:
                {chr(10).join([f"- {ans}" for ans in context_answers])}
                """}
            ],
            temperature=0.3,
        )

        bot_response = response.choices[0].message.content
        chat_history.append({"role": "assistant", "content": bot_response})  # Save bot response
        return bot_response

    except Exception as e:
        return f"Error: {str(e)}"

# Interactive chat loop
def chat():
    print("🟢 Insurance Chatbot Started! (Type 'exit' to end chat)\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("🔴 Chat ended. Goodbye!")
            break
        
        response = get_insurance_answer(user_input)
        #print(f"Bot: {response}\n")
        display(Markdown(f"Bot: {response}\n"))

# Start chatbot
chat()


🟢 Insurance Chatbot Started! (Type 'exit' to end chat)



Bot: Automobile insurance typically covers various expenses related to accidents, damage, and liability. The specific coverage depends on the policy type and optional add-ons selected. Here’s a breakdown of common expenses covered:

### 1. **Liability Coverage**  
   - **Bodily Injury Liability**: Covers medical bills, rehabilitation, and legal fees for others injured in an accident you cause.  
   - **Property Damage Liability**: Pays for repairs or replacement of other parties’ vehicles or property (e.g., fences, buildings) damaged in an accident you cause.  
   - *Required in most states*.

### 2. **Collision Coverage**  
   - Repairs or replaces **your own vehicle** after a collision, regardless of fault (e.g., accidents with another car, hitting a tree).  
   - *Optional but often required for financed/leased vehicles*.

### 3. **Comprehensive Coverage**  
   - Covers non-collision damage to your vehicle from events like:  
     - Theft, vandalism, fire, hail, floods, falling objects.  
     - Animal collisions (e.g., hitting a deer).  
   - *Optional but may be required for leased vehicles*.

### 4. **Medical Payments (MedPay) / Personal Injury Protection (PIP)**  
   - **MedPay**: Covers medical expenses for you and passengers, regardless of fault.  
   - **PIP**: Broader coverage including medical bills, lost wages, and even funeral costs (varies by state).  
   - *Required in "no-fault" states*.

### 5. **Uninsured/Underinsured Motorist Coverage**  
   - Pays for your expenses if the at-fault driver has no insurance or insufficient coverage.  
   - Includes medical bills, vehicle repairs, and sometimes lost wages.  
   - *Required in some states*.

### 6. **Additional Optional Coverages**  
   - **Towing and Labor**: Reimburses for roadside assistance or towing.  
   - **Rental Car Reimbursement**: Covers a rental vehicle while yours is repaired.  
   - **Gap Insurance**: Pays the difference between your car’s value and loan balance if totaled (for leased/financed cars).  
   - **Custom Parts/Equipment**: Covers aftermarket upgrades (e.g., stereo systems, custom wheels).  

### Exclusions (Typically Not Covered):  
   - Intentional damage, normal wear and tear, or mechanical failures.  
   - Driving under the influence (DUI) or illegal activities.  

### Key Factors:  
   - **Deductibles**: The out-of-pocket amount you pay before coverage kicks in (e.g., $500 deductible).  
   - **Policy Limits**: Maximum amounts your insurer will pay per incident (e.g., $50,000 for property damage).  

Always review your policy details and state requirements to ensure adequate coverage. Let me know if you’d like help understanding specific terms or scenarios!


Bot: If you want coverage for expenses due to **self-inflicted damage** to your car (e.g., accidentally backing into a pole, scraping your car in a garage, or other at-fault incidents), here’s what you need to know:

---

### **Coverage Options for Self-Damage**
1. **Collision Coverage**  
   - **What it covers**: Repairs or replacement of **your own vehicle** if you damage it in an accident, even if you’re at fault. Examples:  
     - Hitting a tree, guardrail, or parked car.  
     - Scraping your car while parking.  
   - **Exclusions**: Intentional damage, normal wear and tear, or mechanical failures.  
   - **Required?** Optional, but often mandated for leased/financed vehicles.  

2. **Comprehensive Coverage**  
   - **What it covers**: Non-collision damage caused by **your own mistakes**, such as:  
     - Driving into a flooded area and hydrolocking the engine.  
     - Accidentally setting your car on fire.  
   - **Exclusions**: Damage from intentional acts (e.g., deliberately keying your own car).  

---

### **Key Considerations**
- **Deductible**: You’ll pay this amount out of pocket before insurance kicks in (e.g., a $500 deductible).  
- **Intentional Damage**: Never covered. For example, intentionally ramming your car into a wall won’t be reimbursed.  
- **Wear and Tear**: Routine maintenance (e.g., engine failure from poor upkeep) is excluded.  

---

### **Optional Add-Ons for Extra Protection**
- **Roadside Assistance**: Covers towing if your car breaks down due to self-inflicted issues (e.g., running out of gas).  
- **Original Equipment Manufacturer (OEM) Parts**: Ensures repairs use factory parts instead of cheaper alternatives.  

---

### **What to Do If You Cause Damage**
1. File a claim under **collision coverage** if the damage resulted from an accident.  
2. For non-collision incidents (e.g., fire or flooding), use **comprehensive coverage**.  
3. Avoid filing claims for minor damage if repair costs are close to your deductible.  

---

### **Exclusions**  
- Intentional acts (e.g., vandalizing your own car).  
- Damage from illegal activities (e.g., street racing).  
- Normal wear and tear (e.g., worn-out brakes).  

---

Always review your policy’s terms and discuss specifics with your insurer to ensure you’re adequately covered for accidental self-damage. Let me know if you need help understanding your current policy! 🚗


🔴 Chat ended. Goodbye!
